In [29]:
import numpy as np
import os
import matmul as mm
import math_util as mu
import scipy.io as io
from amm_methods import *
import socket # Obtain the current host name, which can be used to select different data directories and result saving directories

In [30]:
method = METHOD_MITHRAL
# method = METHOD_PQ

In [31]:
feedback_bits = 64
ncodebooks=64
ncentroids=256


In [32]:
host_name = socket.gethostname()
if host_name == 'DESKTOP-PLRL7TK':
    dir_train = ''
    dir_result = ''
elif host_name == 'DESKTOP-6FOH47P':
    dir_train = ''
    dir_result = ''
    data_to_fcpath_train= ''
    featurepath_train= ''
    data_to_fcpath_test = ''
    featurepath_test = ''
elif host_name == 'jm-System-Product-Name':
    dir_train = '/home/hdr/transformer_data/joined'
    dir_result = '/home/hdr/pq/res'
    data_to_fcpath_train= 'data_to_fc_f%i_train.npy' % feedback_bits
    y_train = 'y_f%i_train.npy' % feedback_bits
    featurepath_train= 'feature_f%i_train.npy' % feedback_bits
    data_to_fcpath_test = 'data_to_fc_f%i_test.npy' % feedback_bits
    featurepath_test = 'feature_f%i_test.npy' % feedback_bits
    y_test = 'y_f%i_test.npy' % feedback_bits
else:
    raise NameError("You are running the script in a new computer, please define dir_train")


weightpath = 'encoder_fcw_f%i.npy' % feedback_bits
biaspath = 'encoder_fcb_f%i.npy' % feedback_bits



In [33]:
est3 = mm.estFactory(X_path=data_to_fcpath_train, W_path=weightpath, Y_path=y_train, dir= dir_train, ncodebooks=ncodebooks, ncentroids=ncentroids, methods=[method])


running method:  Mithral
learn_multisplits(): initial loss:    420.3125360567696
learn_multisplits(): returning loss:  170.46519468958843
learn_multisplits(): initial loss:    97.08919403999599
learn_multisplits(): returning loss:  66.05978601994636
learn_multisplits(): initial loss:    1850.8585199694396
learn_multisplits(): returning loss:  717.1428710779451
learn_multisplits(): initial loss:    2363.8082468123366
learn_multisplits(): returning loss:  1243.6454652000634
learn_multisplits(): initial loss:    635.323977701418
learn_multisplits(): returning loss:  285.3327842578398
learn_multisplits(): initial loss:    137.45779233511234
learn_multisplits(): returning loss:  106.17936122722261
learn_multisplits(): initial loss:    2347.0740751496082
learn_multisplits(): returning loss:  1050.9285663284763
learn_multisplits(): initial loss:    2685.082165520218
learn_multisplits(): returning loss:  1513.0226764678955
learn_multisplits(): initial loss:    356.51699749936967
learn_multispl

/home/hdr/pq/bolt/experiments/python/clusterize.py:1436: UserWarning: Persisting input arguments took 1.87s to run.
If this happens often in your code, it can cause performance problems 
(results will be correct in all cases). 
The reason for this is probably some large input arguments for a wrapped
 function (e.g. large strings).
THIS IS A JOBLIB ISSUE. If you can, kindly provide the joblib's team with an
 example so that they can fix the problem.
  _learn_mithral_initialization(X, ncodebooks, pq_perm_algo='start')


X_res mse / X mse:  0.48007688
fitting dense lstsq to X_res
X_res mse / X mse after lstsq:  0.30092287


/home/hdr/pq/bolt/experiments/python/vquantizers.py:610: UserWarning: Persisting input arguments took 1.87s to run.
If this happens often in your code, it can cause performance problems 
(results will be correct in all cases). 
The reason for this is probably some large input arguments for a wrapped
 function (e.g. large strings).
THIS IS A JOBLIB ISSUE. If you can, kindly provide the joblib's team with an
 example so that they can fix the problem.
  self.splits_lists, self.centroids = clusterize.learn_mithral(


In [34]:
x_test = np.load(dir_train+'/'+data_to_fcpath_test)
w_test = np.load(dir_train+'/'+weightpath)
bias = np.load(dir_train+'/'+biaspath)
# print(type(est3))
y_out_matmul = mm.eval_matmul(est3, x_test, w_test) # MADDNESS乘法的结果
# y_out_last = mu.softmax(y_out_matmul + bias.T) # MADDNESS替换后当前层输出，即+bias并激活函数后的结果
y_out_last = y_out_matmul + bias.T # MADDNESS替换后当前层输出，即+bias并不需要激活函数后的结果

In [35]:
print(y_out_last)
print(y_out_last.shape)
# np.save("LDPC_decoder_NET_testdata/" + snr + "nomul_matmul_yout_matmul", y_out_matmul)
# np.save(dir_result+'/'+method+'fc1_fb256_cb%i_ct%i.npy' % (ncodebooks, ncentroids), y_out_matmul)
np.save(dir_result+'/'+method+'fc1_fb%i_cb%i_ct%i.npy' % (feedback_bits, ncodebooks, ncentroids), y_out_last)
# io.savemat(dir_result+'\\fc1_256.mat', {"NN_output_buffer": y_out_last})

[[ 0.11261847 -0.05474875  0.39857587 ... -0.07012872 -0.08791453
  -0.42581087]
 [-0.26311538  0.82196355 -0.35289183 ...  0.30560514 -0.21315914
  -0.17532165]
 [-0.01262615  0.19574049  0.77430975 ...  1.0570729   0.41306394
   0.32565683]
 ...
 [-0.01262615  0.07049587 -0.22764722 ...  0.05511589  1.1645317
  -0.17532165]
 [-0.38836     0.07049587 -0.1024026  ... -0.32061794  0.03733009
  -0.05007704]
 [ 0.3631077   0.19574049 -0.22764722 ... -0.82159644  0.03733009
   0.45090145]]
(32000, 64)
